In [19]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import math 

In [166]:
cap = cv2.VideoCapture('swimming_pool.mov')

ret ,frame = cap.read()
h,w,_ = frame.shape # (height x width x 3)
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # DIVX, XVID, MJPG, X264, WMV1, WMV2,...
out = cv2.VideoWriter('output.mov',fourcc, fps, (w*2,h))

# prepare logo for indicating underwater
underwater_logo = cv2.imread('underwater_logo.png')
logo_h ,logo_w,_ = underwater_logo.shape

left_arrow = cv2.imread('left_arrow.png')
left_arrow_h ,left_arrow_w, _ = left_arrow.shape

right_arrow = cv2.imread('right_arrow.png')
right_arrow_h ,right_arrow_w, _ = right_arrow.shape

# lane mask 
lower = np.array([106,0,105])
upper = np.array([110,255,190])

# region mask
points = np.array([[[int(w/5),h],[int((2/5) * w),int(h/5)],[int((3/5) * w),int(h/5)],[w-int(w/5),h]]])

# dilation kernel 
kernel = np.ones((5,5),np.uint8)

# vertcal filter
vertical_filter = np.array([[1,0,1],
                            [1,0,1],
                            [1,0,1]])/6

while True:
    ret, frame = cap.read()
    if ret==True:

        # check if we are currently underwater using hue values (underwater mainly blue)
        img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        hue,saturation,values = cv2.split(img_hsv)
        (values,counts) = np.unique(hue,return_counts=True)
        ind=np.argmax(counts)
        output_frame = frame.copy()
        
        if 90 <= values[ind] <= 110:
            # we are underwater
            
            # mask lane bounds - clip to / swimming range \
            bounds_mask = np.zeros((h, w), dtype=np.uint8)
            cv2.fillPoly(bounds_mask, points, (255))
            clipped = cv2.bitwise_and(frame,frame,mask = bounds_mask)
            
            # mask lane by hsl - dark blue
            hsv_cropped = cv2.cvtColor(clipped, cv2.COLOR_BGR2HSV)
            color_mask = cv2.inRange(hsv_cropped, lower, upper)
            imask = color_mask>0
            masked_frame = np.zeros_like(frame, np.uint8)
            masked_frame[imask] = frame[imask]
    
            # convert to grayscale
            gray = cv2.cvtColor(masked_frame, cv2.COLOR_BGR2GRAY)

            # convert to black and white using thershold
            _,binary = cv2.threshold(gray,30,255,cv2.THRESH_BINARY)        
        
            # erode noise 
            eroded = cv2.erode(binary,kernel,iterations = 1)
            
             # dilate
            dilated = cv2.dilate(eroded,kernel,iterations = 4)
        
            # run a filter to enhance vertical color changes
            vertical = cv2.filter2D(dilated, -1, vertical_filter)

            # gussian blur
            blured = cv2.GaussianBlur(vertical,(21,21),4)            
        
            # detect edges
            canny = cv2.Canny(blured, 14, 30)
            
            lines = cv2.HoughLines(canny,rho=1,theta=np.pi/90,threshold=65)
            if lines is not None:
                lines_to_draw = []
                for rho,theta in lines[:,0,:]:
                    a = np.cos(theta)
                    b = np.sin(theta)
                    x0 = a*rho
                    y0 = b*rho
                    x1 = int(x0 + 500*(-b))
                    y1 = int(y0 + 500*(a))
                    x2 = int(x0 - 500*(-b))
                    y2 = int(y0 - 500*(a))                   
                    
                    # calculate line angle
                    radians = math.atan2(y1-y0, x1-x0)
                    degrees = math.degrees(radians)
                    if degrees < 0:
                        degrees += 360
                    
                    if (70 <= degrees <= 110) or (250 <= degrees <= 290):
                        lines_to_draw.append(((x1,y1),(x2,y2)))
                        
                if len(lines_to_draw) > 0:
                    left_index = 0
                    right_index = 0
                    min_x = lines_to_draw[0][0][0]
                    max_x = lines_to_draw[0][0][0]
                    for i in range(len(lines_to_draw)):
                        x = lines_to_draw[i][0][0]
                        if x < min_x:
                            left_index = i
                            min_x = x
                        elif x > max_x:
                            right_index = i
                            max_x = x
                            
                    # draw final lines
                    if left_index != right_index:
                        # two lines
                        p01 = lines_to_draw[left_index][0]
                        p02 = lines_to_draw[left_index][1]
                        p11 = lines_to_draw[right_index][0]
                        p12 = lines_to_draw[right_index][1]
                        cv2.line(output_frame,p01,p02,(0,255,0),3)
                        cv2.line(output_frame,p11,p12,(0,255,0),3)

                        # color lane fill
                        pts = np.float32([p01,p02,p11,p12]).reshape(-1,1,2)
                        fill = cv2.fillPoly(np.zeros_like(output_frame),[np.int32(pts)],(255,0,0))
                        output_frame = cv2.addWeighted(output_frame,1,fill,0.5,0)
                    else:
                        # one line
                        p01 = lines_to_draw[left_index][0]
                        p02 = lines_to_draw[left_index][1]
                        cv2.line(output_frame,p01,p02,(0,255,0),3)

                    # check if swimmer is getting far from lane 
                    if max_x < int(w/2):
                        # add sign to turn left
                        output_frame[60 + logo_h: left_arrow_h + 60 + logo_h, 30:+ left_arrow_w + 30] = left_arrow
                    elif min_x > int(w/2):
                        # add sign to turn right
                        output_frame[60 + logo_h: right_arrow_h + 60 + logo_h, 30:+ right_arrow_w + 30] = right_arrow
            
            # add underwater logo to original image
            output_frame[30:logo_h + 30, 30:logo_w + 30] = underwater_logo
            cv2.imshow('target',output_frame)
        else:  
            cv2.imshow('target',frame)
        
        
        final = np.zeros((h, w * 2,3), np.uint8)
        final[:, :w] = frame
        final[:, w:] = output_frame
        out.write(final) # write frame to output file (must be in BGR)
        
        if (cv2.waitKey(1) & 0xff == 27): # ESC key pressed?
            break
    else:
        break

cap.release() # release input video
out.release() # release output video 
cv2.destroyAllWindows() # delete output window
cv2.waitKey(1);